In [ ]:
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#path = 

In [ ]:
#df = pd.read_csv(path)
#df

Per sostituire i Nan

In [ ]:
# df = df.fillna(0) 
# df

Prova

In [ ]:
#data = {'Text': ["Hello, 123 @!£$%' \''s    \n hi", "456 World!", "789 IFOAloveU"," ciao http://www.google.com"]}
#df = pd.DataFrame(data)
#df

Vedere le prime 10 righe

In [ ]:
df['Text'].head(10)

In [ ]:
# Definisci i pattern della REGEX


patterns = {
            # r'\d+': '',             # remove digits (numeri)
            
            # r'[^\w\s]': '',         # remove punteggiatura e simboli ...,'@!£$%
            # r'\b\w{1,2}\b':'',      # remove all token less than2 characters
            # r'(http|www)[^\s]+':'', # remove website
            # r'\s+': ' '             # sostituisce tutti i multipli spazi con uno spazio
            }

def clean_column(df, column, patterns):
    for pattern, replacement in patterns.items():
        df[column] = df[column].str.replace(pattern, replacement)
        df[column] = df[column].str.lower() # applica il lower
    return df

#solo in preview
clean_column(df, 'Text', patterns)

In [ ]:
#applichiamo la modifica o regex
df = clean_column(df, 'Text', patterns)
df

In [ ]:
df['text'][0] #vedi la prima riga
#df['text'].head(10) prime 10 righe

Mostra le classi


In [ ]:
df['target'].value_counts() #tra [] si mette la colonna di cui si vuole sapere le classi

info per vedere se sono convertite in object

In [ ]:
df.info()

Trasformi in stringa se hai solo 1 classe

In [ ]:
text = list(map(str, df['text'])) #trasformi la collonna in stringa 
text = ''.join(text)

Rinonimare le colonne e splittarle in 2

In [ ]:
mapping={0:'NOT DISASTER', 1: 'DISASTER'}
#ovviamente queste sono 2 classi 
df['labels']=df['target'].map(mapping) #target è la colonna che voglio modificare

Si potrebbe fare una wordcloud positiva e una negativa perchè ho 2 classi
(questo NON ha le colonne divise)

In [ ]:
text_neutral = list(map(str, df[df['sentiment']=='neutral']['text']))
text_positive = list(map(str, df[df['sentiment']=='positive']['text']))
text_negative = list(map(str, df[df['sentiment']=='negative']['text']))

text_neutral = ''.join(text_neutral)
text_positive = ''.join(text_positive)
text_negative = ''.join(text_negative)

In [ ]:
## STEP 2 : EDA con WORDCLOUD
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

wordcloud = WordCloud(background_color = 'black', width = 800, height = 400,
                      max_words = 180, contour_width = 3,
                      max_font_size = 80, contour_color = 'steelblue',
                      stopwords = STOPWORDS, random_state = 667)

wordcloud.generate(text)
#wordcloud.generate(text_neutral) o altri paramentri

plt.figure(figsize=(18,10))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")

Per farti veder solo le colonne selezionate, elimini le altre

In [ ]:
#df = df[['text', 'sentiment']]
#df

Elimino i valori nulli 

In [ ]:
df.isnull().sum()

In [ ]:
#df = df.dropna()

In [ ]:
##STEP 3: Definizre X e y (features testuali e Target)

X = df['text']
y = df['class']
print(X.shape)

In [ ]:
##STEP 4: Vettorizzazione
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
vectorizer = CountVectorizer(stop_words='english')
X_vect = vectorizer.fit_transform(X).toarray()
X_vect

In [ ]:
#X_vect.shape

TF = Termini di frequenza, IF = logaritmo. Quindi conteggia il peso delle parole attribuendo più valore alle parole meno frequenti

In [ ]:
##STEP 5: TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X_tfidf = tfidfconverter.fit_transform(X_vect).toarray()
X_tfidf

In [ ]:
##STEP 6: Split Training and Test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y,
                                                    test_size=0.3, 
                                                    random_state=667
                                                    )

Modelli di classificazione

In [ ]:
##STEP 7: Modellazione
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=667,max_depth=5)
classifier.fit(X_train, y_train) 
# provo Random Forest
# accuracy score on the test data
y_pred = classifier.predict(X_test)
test_data_accuracy = accuracy_score(y_pred, y_test)
test_data_accuracy

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train) 
#provo il Multinomiale
# accuracy score on the test data
y_pred = classifier.predict(X_test)
test_data_accuracy = accuracy_score(y_pred, y_test)
test_data_accuracy

In [ ]:
# accuracy score on the test data
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='log', penalty='l2',alpha=1e-3, random_state=667, max_iter=5, tol=None)
classifier.fit(X_train, y_train) 

y_pred = classifier.predict(X_test)
test_data_accuracy = accuracy_score(y_test,y_pred)
test_data_accuracy

Classification report (fare sotto il modello migliore)

lezione 10 conf.matrix.pdf per capire valori precision, recall, f1 score ..

In [ ]:
#labels = ['Class0', 'Class1'] #questi valori sono da cambiare con i nomi delle classi

# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
# questa è quella generica e inserisce lui le classi 

Confusion matrix

In [ ]:
#labels = ['Class0', 'Class1'] questi valori sono da cambiare con i nomi delle classi

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix 

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (10,8))
sns.heatmap(cm,cmap= "Blues", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = classifier.classes_, 
            yticklabels = classifier.classes_)

plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
# Step 8: Pipeline Finale (nuovo train test split su dati raw tetuali) à vuole i dati grezzi 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=667
                                                    )


from sklearn.pipeline import Pipeline

bow = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
tfidf = TfidfTransformer() #inserire modello migliore scelto
#clf =SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=667, max_iter=5, tol=None)
#clf = classifier = MultinomialNB(alpha=0.1) #inserire modello migliore scelto dopo clf
#clf = classifier = RandomForestClassifier(n_estimators=1000, random_state=667,max_depth=5)
pipe = Pipeline([
                ('bow',bow),
                ('tfidf',tfidf),
                ('clf',clf),
                ])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

test_data_accuracy = accuracy_score(y_test,y_pred)
print('Accuracy score of the test data : ', test_data_accuracy)

Classification report sul modello scelto nella pipiline

In [ ]:
from sklearn.metrics import classification_report
#labels = ['class1', 'class2']
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix 

# cm = confusion_matrix(y_test, y_pred)
# plt.figure(figsize = (10,8))
# sns.heatmap(cm,cmap= "Blues", 
#             linecolor = 'black', 
#             linewidth = 1, 
#             annot = True, 
#             fmt='', 
#             xticklabels = classifier.classes_, 
#             yticklabels = classifier.classes_)

# plt.xlabel("Predicted")
# plt.ylabel("Actual")

Cross validation (il test è splittato su più parti del testo)

In [ ]:
# Step 10: Cross Validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe, X, y, scoring = 'f1_micro', cv = 8)

print(f'scores={scores}')
print(f'mean={np.mean(scores)}')
print(f'std={np.std(scores)}')

Salvataggio del modello

In [ ]:
import joblib
joblib.dump(pipe,'NLPEs1.pkl')

In [ ]:
import joblib

uploaded_model = joblib.load('NLPEs1.pkl')
pred = uploaded_model.predict(['very beautiful']) # 1 --> Diabetic sono esempi
pred[0]